## Assignment 3

### 1. Implementation of the EM-Algorithm

In [6]:
import numpy as np

In [7]:
data = np.loadtxt(open("2d-em.csv", "rb"), delimiter=",", skiprows=1)
print(data)

[[ 4.00736258  3.71590642]
 [ 2.49273928  2.5063457 ]
 [ 1.21821548  0.82668865]
 ...
 [-0.59834098 -2.99436545]
 [-3.64272931  0.79219991]
 [-2.10190378 -1.9862288 ]]
